## PointHRNet
Uses the High Resolution Net (HRNet) from Microsoft Research Asia to predict 'heat maps' for key points. Input data is a grayscale image and the input label for a datapoint is two numbers (x,y) that have been normalized to fit on [0,1] corresponding to the location on the input image. This input label is then transformed into an image (the same size as the data) with a 2D Gaussian centered at the key point (with a custom standard deviation, typically around 1 pixel). One can have multiple key points per image, which will all get their own heat maps.

## Import
Using PyTorch and Torchvision, Image Processing SciKit, Matplotlib, Time, Sys, Glob, OS, Math, Random, HRNet (leoxiaobin), NumPy, OpenCV, Itertools

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from skimage import io, transform
import matplotlib.pyplot as plt
import time
import sys
import glob
import os
import math
import random
from pose_hrnet import PoseHighResolutionNet
import numpy as np
import cv2
import itertools

## Constants/Parameters
Constants/parameters that can be changed (although most variables must reflect the input data parameters, like image size, or the model and loss function parameters, like the GPU data types). Data is in a 1x1 image format and the labels are in a text file with the name of the file on a line, then a (x,y) location point below it, then finally this is transformed into an image heat map with a 2D Gaussian around the (x,y) location. If there are more than one location points being trained, these are on seperate lines.

#### Note 1:
If not loading all images into memory on the creation of the dataset class, it is fastest to use a 1 image x 1 image grid. Should be fine on the OS unless using > ~1 million images. If loading to RAM, should be fine using a larger grid, however
the initial process of loading will be slower (the statement in this sentence needs to be tested).

#### Note 2:  
The fastest method seems to be storing the data in ram (STORE_DATA_RAM = True) and keeping pinned memory off (PIN_MEMORY = False) and no multithreading on the CPU (NUM_WORKERS = 0). However, if RAM storage is running out due to a very large dataset, then turn off storing data in RAM (STORE_DATA_RAM = False), turn on pinned memory (PIN_MEMORY = True), and add multi-threading (NUM_WORKERS = 2 seems to be a sweet spot although this could vary and should be experimented with if RAM storage is turned off). This won't be as fast as using the RAM storage method with no pinning and no multithreading, but it should still be decently fast. If running with multi-threading, don't use the Jupyter enviroment - instead run straight from Python.

#### Note 3:  
The GPU tensor data types are based on the input to the model (architecture) and the input to the loss function (which is a function of the output of the model and the labels). Make these compatible with the model (architecture) input and loss function requirements to avoid slow, implicit recasts (at least I think that is what's happening...). These types should be experimented with (or alternatively go read the documentation) to find the appropriate (which should also be the fastest) types when using a different architecture and/or loss function.

In [ ]:
# Data constants
IMAGE_HEIGHT = 512
IMAGE_WIDTH = 512
DATA_HOME_DIR = 'C:/Users/pflood/Desktop/JTAML/Python/PyTorch/PointHRNet/dataEmil'
MODEL_TYPE = 'points'

# Data loader parameters
BATCH_SIZE = 16
SHUFFLE = True
NUM_WORKERS = 0 # If not storing data in RAM, try turning this on (2 is potentially a good number)
PIN_MEMORY = False # Slower if storing in RAM (so keep False), but seems to make faster if using workers and not storing in RAM 

# Data storage parameters
'''
STORE_DATA_RAM loads images and labels to ram on creation of dataset class if True.
Much faster if room for it. Keep NUM_WORKERS = 0 and PIN_MEMORY = False, if turned on (STORE_DATA_RAM = True)
'''
STORE_DATA_RAM = True

# GPU Tensor Data Types
'''
These are the data types for the images and their corresponding labels when both are pushed to the GPU(s).
Make these compatible with the model (architecture) input and loss function requirements to avoid slow, implicit recasts.
Might need to play around or read documentation to figure this out when changing architectures and/or loss function.
'''
IMAGES_GPU_DATA_TYPE = torch.FloatTensor

# Training Parameters
MAX_EPOCHS = sys.maxsize # How many epochs to run training cycle, sys.maxsize gives 2^63 which will essentially run forever

# Validation Parameters
'''
Don't want too big as this appears to take up additional GPU space, however too small will lead to latencies in code. 4 is good.
'''
VALIDATION_BATCH_SIZE = 4

# Image Printing Parameters
'''
NUM_PRINT_IMG is the number of sampled images (number of rows).
KP_PLOT_RAD is the radius in pixels of the plotted key points.
'''
NUM_PRINT_IMG = 1
KP_PLOT_RAD = 3

# Model name
MODEL_NAME ='EMIL_POINTS_VER_1'

# Loading from previous checkpoint?
LOAD_FROM_CHECKPOINT = True

# Number of channels in input image (Grayscale = 1, RGB = 3, etc.)
IMG_CHANNELS = 1

# Number of Feature Points in input image (assumes all images have this many points)
NUM_POINTS = 23

# Standard Deviation for 2D Gaussian Heat Map Label (in pixel units on a Euclidean plane)
GAUSSIAN_STDEV_HEATMAP = 5
GAUSSIAN_AMP = 1e6

# Test Output Original Directory from C++
TEST_OUTPUT_DIR = 'C:/Users/pflood/Desktop/JTAML/RealShoulderData/AkiraOutput43PointsLeftScapulaTest'

# Test Original Directory from C++
TEST_ORIGINAL_DIR = 'C:/Users/pflood/Desktop/JTAML/RealShoulderData/AkiraOrganizedLeftTest'

# Crop to Specified Range on Images (Way of Feeding In Higher Resolution Images)
CROP_IMAGES = False
CROP_MIN_X = 0.29
CROP_MAX_X = 0.84
CROP_MIN_Y = 0.45
CROP_MAX_Y = 0.95

#### Constants/Parameters Dictionaries
Dictionaries of certain parameters and constants. This does not have to be edited if solely adjusting the values of the above constants/parameters.

In [ ]:
# Create a dictionary of the constants (IMAGES PER GRID KEPT CONSTANT AT 1)
data_constants = {'image_height':IMAGE_HEIGHT, 'image_width':IMAGE_WIDTH,\
            'per_grid_image_count_height':1, 'per_grid_image_count_width':1,\
            'images_per_grid':1,
            'data_home_dir':DATA_HOME_DIR, 'model_type':MODEL_TYPE}

# Dictionary of parameters to be fed to data loader
data_loader_parameters = {'batch_size': BATCH_SIZE, 'shuffle': SHUFFLE,'num_workers': NUM_WORKERS,'pin_memory': PIN_MEMORY}

## Create Gaussian Heatmaps
Create gaussian heat maps centered at points with GAUSSIAN_STDEV_HEATMAP standard deviation.

In [ ]:
def create_gaussian_heatmap(point_list, img_h, img_w):
        '''
        Takes list of (x,y) points and returns a list of gaussian heat maps centered at the respective points.
        '''
        heat_maps = []
        for point in point_list:
            g_x = np.linspace(0.5, img_w - 0.5, img_w)
            g_y = np.linspace(0.5, img_h - 0.5, img_h)
            g_x, g_y = np.meshgrid(g_x, g_y)
            g_z = GAUSSIAN_AMP/(2*np.pi*GAUSSIAN_STDEV_HEATMAP**2) *np.exp(
                #-(((g_x - math.floor(point[0]*img_w)+0.5)**2 + (g_y - math.floor(
                #    img_h*(1 - point[1]))+0.5)**2) /(2*GAUSSIAN_STDEV_HEATMAP**2))) # ADDED FLOOR TO MAKE SURE GAUSSIAN ALWAYS PEAKS ON PIXEL
                -(((g_x - point[0]*img_w)**2 + (g_y - img_h*(1 - point[1]))**2) /(2*GAUSSIAN_STDEV_HEATMAP**2)))
            heat_maps.append(g_z)
        return np.stack( heat_maps, axis=0 )

## PyTorch Feature Point Dataset Class
Dataset class for point detection problems where images are arranged in 1x1 grids.

#### Note 1:
The model is fit on a training dataset. This trained model is used to predict responses in a validation dataset.
This gives an unbiased evaluation of the model fitted on the training dataset, and we can adjust the hyperparameters (which
includes parameters like batch size or even the types of images used as training data) to maximize performance on the validation
dataset. To get an unbiased evaluation with respect to the hyperparameters we further evaluate on a test dataset. However,
note that we save the model which peforms best on the validation dataset. The test dataset purely gives us a final evaluation. we denote the three evaluation type classes as: 'training', 'validation', and 'test'.

#### Note 2:
It is assumed that the labels sit in a file in a seperate folder found at %DATA_HOME_DIR%/labels/%MODEL_TYPE%. For example, on a "fem" model, the label file will be at %DATA_HOME_DIR%/labels/fem and called fem_labels.txt. Note that the corresponding images are further categorized into folders based on evaluation type. It is also assumed that all grids have the same image width, image height, grid width of 1, grid height of 1, and model type (i.e. the same data constants). Note that each image grid should appear in exactly one of the three different evaluation type directories (e.g. we don't use an image in the validation set if it already appears in the training set). These directories are given by %DATA_HOME_DIR%/%evaluation type%. Different grids must have different names even across different evaluation types. For example, we cannot have a grid45.tif in both the training set and test set, even if both grid45.tif files contain different images.In the label file, the image name will take up a line, then for each NUM_POINTS there will be a line after the image name that corresponds to the location of a feature point. Across different images this order is maintained (i.e. the ith point after an image is the same type of feature, say humerus head, across all images). The image grids only need to have unique names and can be called anything (although they need to be .tif files). 

#### Note 2:
The __getitem__ method always returns the image and label tensors in the data type that the neural network and loss function need. If storing in RAM the images and labels are stored as bytes in RAM and then when __getitem__ is called they are cast to their required data types. If not storing in RAM, when the images and labels are generated, they are assigned their required data types.

In [ ]:
class FeaturePointDataset(Dataset):
    """
    Feature Point Dataset.
    1. Grayscale images are stored in grids to reduce the number of file paths.
    2. Labels are read in as NUM_POINTS 2-tuples per image stored in a text file, then transformed to 2D Gaussian heat maps.
        a). Labels should be normalized in the text file already.
        b). The label text file is stored in the %data_home_dir%/labels/%model_type% folder and is always named
        '%model_type%_labels.txt'.
        c). Checks will be performed to ensure that each 1x1 grid in %data_home_dir%/%evaluation_type% has a corresponding label, 
        else the class will throw an exception.
    """
    
    def __init__(self, data_constants, store_data_ram, evaluation_type, num_points, transform=None):
        """
        Args:
            data_constants (dictionary): Dictionary of vital constants about data.
            store_data_ram (boolean): Choose to store data in RAM for faster processing, or save RAM at a cost in performance.
            evaluation_type (string): Dataset evaluation type (must be 'training', 'validation', or 'test')
            num_points (int): Number of feature points per image.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        # Local copy of data_constants
        self.data_constants = data_constants
        
        # Local copy of num_points
        self.num_points = num_points
        
        # Check that evaluation_type is valid and then store
        if evaluation_type in ['training', 'validation', 'test']:
            self.evaluation_type = evaluation_type
        else:
            raise Exception('Incorrect evaluation type! Must be either \'training\', \'validation\', or \'test\'.')
        
        # Create full paths to all grids and their Labels
        '''
        Search %data_home_dir%/%evaluation_type%/ for all .tif files. 
        Check that properly named label text file in %data_home_dir%/labels/%model_type%/ exists for the given model type.
        Throw an exception if this is not the case.
        '''
        image_grids = glob.glob1(self.data_constants['data_home_dir'] + '/' + self.evaluation_type, "*.tif") # Just name and .tif extension
        self.grids_fullpaths = [self.data_constants['data_home_dir']  + '/' + self.evaluation_type + '/' +\
                      image_grid_name for image_grid_name in image_grids]
            
        # Calculate grid count
        self.grid_count = len(self.grids_fullpaths)
        
        # Check that label text file exists
        self.label_text_file = self.data_constants['data_home_dir']  + '/labels/' + self.data_constants['model_type']\
                                     + '/' + self.data_constants['model_type'] + '_KPlabels.txt'
        if os.path.isfile(self.label_text_file) == False:
            raise Exception('Error, cannot find file: ' + self.label_text_file)
        
        # Read in n x num_points x 2 list of point locations and n length list of associated file names
        raw_text_list = [line.rstrip('\n') for line in open(self.label_text_file)]
        # Check that the raw text_list has at least length (num_points + 1)*self.grid_count
        if len(raw_text_list) < (num_points + 1)*self.grid_count:
            raise Exception('Error, the text file only contains ' + str(len(raw_text_list)) + ' lines!')
        # Take Out The File Names
        text_filenames = raw_text_list[0::(num_points + 1)]
        del raw_text_list[0::(num_points + 1)]
        # Organize Data in Same Order as Image Grids, If No Match Throw Error
        label_str_data = []
        data_point_strings = [line.split(',') for line in raw_text_list]
        for image_scroll_idx in range(0,len(image_grids)):
            for text_fname_idx in range(0,len(text_filenames)):
                if image_grids[image_scroll_idx] == text_filenames[text_fname_idx]:
                    label_str_data.append(data_point_strings[(text_fname_idx*num_points):(text_fname_idx*num_points + num_points)])
                    break
                if text_fname_idx == (len(text_filenames) - 1):
                    raise Exception('Error, could not find label for ' + image_grids[image_scroll_idx] + '!')
        # Convert to Numpy Array
        self.label_point_data = np.array(label_str_data, dtype=float)
        
        # Error check:
        # self.grid_fullpaths is the data input (images) of length 'n' and self.label_point_data are the labels
        # Make sure self.label_point_data has size n x num_points x 2
        if self.label_point_data.shape != (len(self.grids_fullpaths),num_points,2):
                     raise Exception('Error, label data array has shape ' + str(self.label_point_data.shape) + '!')
        
        # Optional transform
        self.transform = transform
        
        # Store image tensors and label tensors to CPU RAM option (should be faster as long as there is room in the RAM)
        self.store_data_ram = store_data_ram
        self.data_storage = []
        if self.store_data_ram:
            for idx in range(self.grid_count*self.data_constants['images_per_grid']): # Total number of images
                self.data_storage.append(self.read_in_data(idx))
                
        # Print successful initialization message
        print ("Successfully initialized " + self.evaluation_type + ' dataset!')

    def __len__(self):
        return self.grid_count*self.data_constants['images_per_grid'] # Total number of images in data type (n)
    
    def read_in_data(self, idx):
        # Read in image grid
        grid_idx = idx//self.data_constants['images_per_grid']
        grid_image = io.imread(self.grids_fullpaths[grid_idx], as_gray=True)
        
        # Extract image from grid using top-left to bottom-right ordering
        idx_in_grid = idx%self.data_constants['images_per_grid']
        img_top_row_idx = (idx_in_grid//self.data_constants['per_grid_image_count_width'])*self.data_constants['image_height']
        img_left_col_idx = (idx_in_grid%self.data_constants['per_grid_image_count_width'])*self.data_constants['image_width']
        image = grid_image[img_top_row_idx:img_top_row_idx + self.data_constants['image_height'],\
                          img_left_col_idx:img_left_col_idx + self.data_constants['image_width']]
        
        # Label should always be in [0,1] format when read in and then transformed into Gaussian heatmap       
        # In PyTorch, images are represented as [channels, height, width] so must add 1 channel dimension
        if CROP_IMAGES:
            LEFT_X_PIX = math.floor(CROP_MIN_X*IMAGE_WIDTH)
            RIGHT_X_PIX = math.ceil(CROP_MAX_X*IMAGE_WIDTH)
            LEFT_X_PIX -= (32 - ((RIGHT_X_PIX - LEFT_X_PIX) % 32)) # So Width is Divisible by 32
            BOT_Y_PIX = math.floor(CROP_MIN_Y*IMAGE_HEIGHT)
            TOP_Y_PIX = math.ceil(CROP_MAX_Y*IMAGE_HEIGHT)
            BOT_Y_PIX -= (32 - ((TOP_Y_PIX - BOT_Y_PIX) % 32)) # So Height is Divisible by 32
            image = torch.ByteTensor(image[None, (IMAGE_HEIGHT - TOP_Y_PIX):(IMAGE_HEIGHT-BOT_Y_PIX), LEFT_X_PIX:RIGHT_X_PIX]) # Store as byte (to save space) then convert
            cropped_point_list = [] # Need to Rescale Points
            for orig_point in self.label_point_data[idx]:
                cropped_point_list.append([(orig_point[0]*IMAGE_WIDTH - LEFT_X_PIX)/(RIGHT_X_PIX-LEFT_X_PIX),\
                                           (orig_point[1]*IMAGE_HEIGHT - BOT_Y_PIX)/(TOP_Y_PIX-BOT_Y_PIX)])
            label = torch.FloatTensor(create_gaussian_heatmap(cropped_point_list, TOP_Y_PIX-BOT_Y_PIX, RIGHT_X_PIX-LEFT_X_PIX))
            #image = torch.zeros([1,TOP_Y_PIX-BOT_Y_PIX, RIGHT_X_PIX-LEFT_X_PIX],dtype=torch.uint8)
            #label = torch.zeros([NUM_POINTS,TOP_Y_PIX-BOT_Y_PIX, RIGHT_X_PIX-LEFT_X_PIX], dtype=torch.float)
        else:
            image = torch.ByteTensor(image[None, :, :]) # Store as byte (to save space) then convert when called in __getitem__
            label = torch.FloatTensor(create_gaussian_heatmap(self.label_point_data[idx], IMAGE_HEIGHT, IMAGE_WIDTH))
        
        # Form sample and transform if necessary
        sample = {'image': image, 'label': label}
        if self.transform:
            sample = self.transform(sample)
        return sample

    def __getitem__(self, idx):
        if self.store_data_ram:
            return {'image': self.data_storage[idx]['image'].type(IMAGES_GPU_DATA_TYPE), 'label':\
                   self.data_storage[idx]['label']}
        else:
            # Read in image grid
            grid_idx = idx//self.data_constants['images_per_grid']
            grid_image = io.imread(self.grids_fullpaths[grid_idx], as_gray=True)

            # Extract image from grid using top-left to bottom-right ordering
            idx_in_grid = idx%self.data_constants['images_per_grid']
            img_top_row_idx = (idx_in_grid//self.data_constants['per_grid_image_count_width'])*self.data_constants['image_height']
            img_left_col_idx = (idx_in_grid%self.data_constants['per_grid_image_count_width'])*self.data_constants['image_width']
            image = grid_image[img_top_row_idx:img_top_row_idx + self.data_constants['image_height'],\
                              img_left_col_idx:img_left_col_idx + self.data_constants['image_width']]
            
            # Label should always be in [0,1] format when read in and then transformed into Gaussian heatmap       
            # In PyTorch, images are represented as [channels, height, width] so must add 1 channel dimension
            if CROP_IMAGES:
                LEFT_X_PIX = math.floor(CROP_MIN_X*IMAGE_WIDTH)
                RIGHT_X_PIX = math.ceil(CROP_MAX_X*IMAGE_WIDTH)
                LEFT_X_PIX -= (32 - ((RIGHT_X_PIX - LEFT_X_PIX) % 32)) # So Width is Divisible by 32
                BOT_Y_PIX = math.floor(CROP_MIN_Y*IMAGE_HEIGHT)
                TOP_Y_PIX = math.ceil(CROP_MAX_Y*IMAGE_HEIGHT)
                BOT_Y_PIX -= (32 - ((TOP_Y_PIX - BOT_Y_PIX) % 32)) # So Height is Divisible by 32
                image = torch.ByteTensor(image[None, (IMAGE_HEIGHT - TOP_Y_PIX):(IMAGE_HEIGHT-BOT_Y_PIX), LEFT_X_PIX:RIGHT_X_PIX]) # Store as byte (to save space) then convert
                cropped_point_list = [] # Need to Rescale Points
                for orig_point in self.label_point_data[idx]:
                    cropped_point_list.append([(orig_point[0]*IMAGE_WIDTH - LEFT_X_PIX)/(RIGHT_X_PIX-LEFT_X_PIX),\
                                               (orig_point[1]*IMAGE_HEIGHT - BOT_Y_PIX)/(TOP_Y_PIX-BOT_Y_PIX)])
                label = torch.FloatTensor(create_gaussian_heatmap(cropped_point_list, TOP_Y_PIX-BOT_Y_PIX, RIGHT_X_PIX-LEFT_X_PIX))
            else:
                image = torch.ByteTensor(image[None, :, :]) # Store as byte (to save space) then convert when called in __getitem__
                label = torch.FloatTensor(create_gaussian_heatmap(self.label_point_data[idx], IMAGE_HEIGHT, IMAGE_WIDTH))
        
            # Form sample and transform if necessary
            sample = {'image': image, 'label': label}
            if self.transform:
                sample = self.transform(sample)
            return sample

## Plot Image/Prediction/Overlay Function
Function that is used to plot a sample of images from the validation batch and the corresponding true key points (in cyan), labels for the true key points (in green), the predicted version of the key points (in red), and a yellow line between the predicted and true key point.

In [ ]:
def plot_predictions(validation_image_batch, validation_label_batch, validation_output_batch, number_images_print):
    '''
    Plots a sample of images from the validation batch and the corresponding true key points (in cyan),
    labels for the true key points (in green), the predicted version of the key points (in red),
    and a yellow line between the predicted and true key point.
    '''
    # Check rows is less than validation set size
    if (number_images_print > len(validation_image_batch)):
        number_images_print = len(validation_image_batch)
        print('Warning: Attempted to print more images than the validation batch contains!')
        print('Only printing',number_images_print,'image(s).')
    sample_image_indices = random.sample(range(0, len(validation_image_batch)), number_images_print)
 
    # Plot Image With Key Points, Labels, Guesses for Key Points, Line between Each Actual KP and Guess KP
    for img_to_print_idx in sample_image_indices:
        plt.figure(figsize=(48,(IMAGE_HEIGHT/IMAGE_WIDTH)*48))
        I = validation_image_batch[img_to_print_idx].to("cpu").type(torch.float32)
        L_heatmaps = (validation_label_batch[img_to_print_idx].to("cpu")).type(torch.float32)
        # Convert Heatmaps to Points By Taking The Maximum For Each Keypoint (Labels)
        L = []
        for kp_ind in range(0,L_heatmaps.shape[0]):
            Lm_ind = torch.argmax(L_heatmaps[kp_ind])
            Lx_ind = Lm_ind.item() % L_heatmaps[kp_ind].shape[1]
            Ly_ind = Lm_ind.item() // L_heatmaps[kp_ind].shape[1] # Indexed by top left corner (same as OpenCV)
            L.append([Lx_ind, Ly_ind])
        G_heatmaps = (validation_output_batch[img_to_print_idx].to("cpu")).type(torch.float32)
        # Convert Heatmaps to Points By Taking The Maximum For Each Keypoint (Guesses)
        G = []
        for kp_ind in range(0,G_heatmaps.shape[0]):
            Gm_ind = torch.argmax(G_heatmaps[kp_ind])
            Gx_ind = Gm_ind.item() % G_heatmaps[kp_ind].shape[1]
            Gy_ind = Gm_ind.item() // G_heatmaps[kp_ind].shape[1] # Indexed by top left corner (same as OpenCV)
            G.append([Gx_ind, Gy_ind])
        img_PIL = torchvision.transforms.ToPILImage()(I.type(torch.uint8))
        img_cv = cv2.cvtColor(np.array(img_PIL), cv2.COLOR_GRAY2BGR)
        for point_idx in range(0,NUM_POINTS):
            cv2.circle(img_cv,((L[point_idx][0]),(L[point_idx][1])), KP_PLOT_RAD, (0,255,255))
            cv2.circle(img_cv,((G[point_idx][0]),(G[point_idx][1])), KP_PLOT_RAD, (255,0,0))
            #cv2.putText(img_cv,str(point_idx),(int(IMAGE_WIDTH*L[point_idx,0].item()),int(IMAGE_HEIGHT* (1 - L[point_idx,1].item()))),0,FONT_SCALE_PLOT,(0,255,0))
            cv2.line(img_cv, ((L[point_idx][0]),(L[point_idx][1])),\
                     ((G[point_idx][0]),(G[point_idx][1])), (255,255,0))
        plt.imshow(img_cv)
        plt.show()
        plt.imshow(L_heatmaps[0].numpy())
        plt.colorbar()
        plt.show()
        plt.imshow(G_heatmaps[0].detach().numpy())
        plt.colorbar()
        plt.show()

## Main Loop
Initialize model and move to the main device (first GPU). Allow multi-GPU processing, and generate a loss function (moved to the main device for safety). Create an optimizer (must be created after moving model to GPU), then generate a data loader and traverse it in a training loop. A data loader should also be created for a validation set. Model weights are saved to the SSD when average validation set error reaches a new low. The entire block of code is wrapped in an if statement that protects multithreading on a Windows 10 OS.

At the end of each forward pass of a batch, the most recent training batch loss is printed. At the end of each epoch, the average loss over the validation set is calculated, and the model's learnable parameters are saved to the SSD if this average loss is the new minimum over all previous average validation losses. If we are saving a new minimum, a sample of validation images with corresponding predicted segmented masks and an overlay of the two will be printed. The epoch training time, current index, average validation loss, and the current minimum average validation loss (along with the corresponding epoch index) are printed regardless of whether or not the average validation loss for the epoch was a new minimum.

In [ ]:
if __name__ == '__main__': 
    # CUDA for PyTorch
    use_cuda = torch.cuda.is_available()
    main_device = torch.device("cuda:0" if use_cuda else "cpu")
    torch.backends.cudnn.benchmark = True # Eventually makes faster after initial computational cost
    
    # Make model and move to main_device
    model = PoseHighResolutionNet(num_key_points = NUM_POINTS, num_image_channels = IMG_CHANNELS)
    model = model.to(main_device)

    # Split Across Multiple Devices
    if torch.cuda.device_count() > 1 and main_device.type == 'cuda':
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(model) # Splits batches across GPUs
        # Name additional devices
        additional_devices = []
        for device_id in range(1,torch.cuda.device_count()):
            additional_devices.append(torch.device("cuda:" + str(device_id)))
 
    # Print info when using cuda
    if main_device.type == 'cuda':
        print(torch.cuda.get_device_name(main_device),main_device,'- Main Device')
        print('Memory Usage:')
        print('Allocated:', round(torch.cuda.memory_allocated(main_device)/1024**3,1), 'GB')
        print('Cached:   ', round(torch.cuda.memory_cached(main_device)/1024**3,1), 'GB')                              
        for extra_dev in additional_devices:
            print(torch.cuda.get_device_name(extra_dev),extra_dev)
            print('Memory Usage:')
            print('Allocated:', round(torch.cuda.memory_allocated(extra_dev)/1024**3,1), 'GB')
            print('Cached:   ', round(torch.cuda.memory_cached(extra_dev)/1024**3,1), 'GB')                              
    
    # Load model weights if loading from previous checkpoint
    if LOAD_FROM_CHECKPOINT:
        checkpoint = torch.load('./'+ MODEL_NAME + '.pth')
        model.load_state_dict(checkpoint['model_state_dict'])
        start_epoch = checkpoint['epoch']
        print('Checkpoint average validation loss:',checkpoint['validation_loss'])
    else:
        start_epoch = -1
        
    # Make loss function and move to device (will run the cuda loss function if input tensor is a cuda tensor, but just in case)
    loss_fn = torch.nn.MSELoss().to(main_device)
    
    # Make Adam optimizer
    # It is recommended to move a model to GPU before constructing an optimizer:
    # (https://pytorch.org/docs/master/optim.html)
    optimizer = torch.optim.Adam(model.parameters())
    if LOAD_FROM_CHECKPOINT:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    # Training Generators
    training_set = FeaturePointDataset(data_constants, STORE_DATA_RAM, 'training', NUM_POINTS)
    training_generator = torch.utils.data.DataLoader(training_set, **data_loader_parameters)
    
     # Validation Generators
    validation_set = FeaturePointDataset(data_constants, STORE_DATA_RAM, 'validation', NUM_POINTS)
    validation_generator = torch.utils.data.DataLoader(validation_set, batch_size=VALIDATION_BATCH_SIZE, shuffle = True)
    
    # Initialize current minimums
    if LOAD_FROM_CHECKPOINT:
        current_minimum_avg_validation_loss = checkpoint['validation_loss']
    else:
        current_minimum_avg_validation_loss = math.inf
    current_minimum_epoch_idx = 0
    
    # Loop over epochs
    for epoch_idx in range(start_epoch + 1, MAX_EPOCHS):
        # Epoch timer
        epoch_timer_start = time.time()
        
        # Training loop for an epoch
        print("-"*65)
        print('BEGIN EPOCH', epoch_idx)
        model.train()
        for training_batch_idx, training_batch in enumerate(training_generator):
            # Transfer to GPU
            training_batch, training_batch_labels = training_batch['image'].to(main_device, dtype=torch.float, non_blocking=True)\
            ,training_batch['label'].to(main_device, dtype=torch.float, non_blocking=True)
            
            # Zero the gradient buffers
            optimizer.zero_grad()
            
            # Forward through the model
            training_output = model(training_batch) # NOT NORMALIZING GRAYSCALE
            
            # Calculate loss
            training_loss = loss_fn(training_output, training_batch_labels)
            print('Epoch', epoch_idx, '--- Batch', training_batch_idx, 'training loss before optimization step:',\
                  training_loss.item())
                
            # Backprop then update using optimizer
            training_loss.backward()
            optimizer.step()
        
        # Calculate average loss over validation set
        total_validation_loss = 0
        model.eval()
        for validation_batch in validation_generator:
            # Transfer to GPU
            validation_batch, validation_batch_labels = validation_batch['image'].\
            to(main_device, dtype=torch.float, non_blocking=True),\
            validation_batch['label'].to(main_device, dtype=torch.float, non_blocking=True)
            
            # Forward through the model
            validation_output = model(validation_batch) # NOT NORMALIZING GRAYSCALE
            
            # Calculate loss and keep a running sum total of the loss over all the validation data
            validation_loss = loss_fn(validation_output, validation_batch_labels)
            total_validation_loss += validation_loss.item()*len(validation_batch)
        average_validation_loss = total_validation_loss/len(validation_set)
        '''
        If the average validation loss is less than the current minimum, update the current minimum.
        Also, print out original image/predicted segmentation image/overlay of pairs for a few samples from the validation set.
        Finally, save the model's learned parameters to the SSD.
        '''
        if average_validation_loss < current_minimum_avg_validation_loss:
            current_minimum_avg_validation_loss = average_validation_loss
            current_minimum_epoch_idx = epoch_idx
            print ('New minimum average validation loss found: ', current_minimum_avg_validation_loss)
            
            # Print out image, predicted segmentation mask, and an overlay
            plot_predictions(validation_batch,validation_batch_labels, validation_output,NUM_PRINT_IMG)
            
            # Save the model
            torch.save({'epoch': epoch_idx,'model_state_dict': model.state_dict(),'optimizer_state_dict': optimizer.state_dict()\
                        ,'validation_loss': current_minimum_avg_validation_loss}, './'+ MODEL_NAME + '.pth')
        elif LOAD_FROM_CHECKPOINT and epoch_idx == (start_epoch + 1): # Print out on first epoch if loading from a checkpoint
            # Print out image, predicted segmentation mask, and an overlay
            plot_predictions(validation_batch, validation_batch_labels, validation_output,NUM_PRINT_IMG)
            current_minimum_epoch_idx = start_epoch

        print('Epoch', epoch_idx, 'has average validation loss:', average_validation_loss)
        print('Current minimum average validation loss:', current_minimum_avg_validation_loss,\
              '(from epoch ' + str(current_minimum_epoch_idx) + ')')
        print ('Epoch', epoch_idx, 'runtime:',time.time() - epoch_timer_start,'seconds')

# Test, Solve PNP, and Produce Kinematics/Evaluations
Predicts 2D key points for an image from the test dataset, then uses OpenCV to solve the PNP problem and determine a rotation/translation 6 DOF pose for the model. Once this is done for all images, comparisons with the true kinematics are performed. Other tests on the accuracy of each point, different PnP solutions over other subsets of the key points.

## Load Test Data
The subection below loads all necessary test data into place.

In [ ]:
if __name__ == '__main__': 
    # CUDA for PyTorch
    use_cuda = torch.cuda.is_available()
    main_device = torch.device("cuda:0" if use_cuda else "cpu")
    torch.backends.cudnn.benchmark = True # Eventually makes faster after initial computational cost
    
    # Make model and move to main_device
    model = PoseHighResolutionNet(num_key_points = NUM_POINTS, num_image_channels = IMG_CHANNELS)
    model = model.to(main_device)

    # Split Across Multiple Devices
    if torch.cuda.device_count() > 1 and main_device.type == 'cuda':
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(model) # Splits batches across GPUs
        # Name additional devices
        additional_devices = []
        for device_id in range(1,torch.cuda.device_count()):
            additional_devices.append(torch.device("cuda:" + str(device_id)))
 
    # Print info when using cuda
    if main_device.type == 'cuda':
        print(torch.cuda.get_device_name(main_device),main_device,'- Main Device')
        print('Memory Usage:')
        print('Allocated:', round(torch.cuda.memory_allocated(main_device)/1024**3,1), 'GB')
        print('Cached:   ', round(torch.cuda.memory_cached(main_device)/1024**3,1), 'GB')                              
        for extra_dev in additional_devices:
            print(torch.cuda.get_device_name(extra_dev),extra_dev)
            print('Memory Usage:')
            print('Allocated:', round(torch.cuda.memory_allocated(extra_dev)/1024**3,1), 'GB')
            print('Cached:   ', round(torch.cuda.memory_cached(extra_dev)/1024**3,1), 'GB')                              
    
    # Load model weights if loading from previous checkpoint
    if LOAD_FROM_CHECKPOINT:
        checkpoint = torch.load('./'+ MODEL_NAME + '.pth')
        model.load_state_dict(checkpoint['model_state_dict'])
        start_epoch = checkpoint['epoch']
        print('Checkpoint average validation loss:',checkpoint['validation_loss'])
    else:
         raise Exception('Must be loading trained model if running in test mode!')
            
     # Make loss function and move to device (will run the cuda loss function if input tensor is a cuda tensor, but just in case)
    loss_fn = torch.nn.MSELoss().to(main_device)
        
     # Test Generators
    test_set = FeaturePointDataset(data_constants, STORE_DATA_RAM, 'test', NUM_POINTS)
    test_generator = torch.utils.data.DataLoader(test_set, batch_size=VALIDATION_BATCH_SIZE, shuffle = False)
    
    '''
    BEGIN READ IN IMAGE INFORMATION:
    Width, Height, True Pose [X,Y,Z,ZA,XA,YA], Calibration File Info [PD,XO,YO,PP], 
    Model Key Points [NUM_POINTS by [X,Y,Z]], Projected Key Points [NUM_POINTS by [X,Y]]
    
    # Read In Each Line of Info File
    info_file = TEST_OUTPUT_DIR + '/info_' + MODEL_TYPE +'.txt'
    info_lines = [line.rstrip('\n') for line in open(info_file)]
    # Double Check That There Are 13*# of Test Files)
    if len(info_lines) != 13*len(test_set):
        raise Exception('Info file length is not 13 x the number of loaded test files!')

    # Read in Projected Key Points File
    # Projected Key Points
    proj_kp_file = TEST_OUTPUT_DIR + '/' + MODEL_TYPE + '_KPlabels.txt'
    proj_kp_lines = [line.rstrip('\n') for line in open(proj_kp_file)]
    # Double Check That There Are NUM_POINTS + 1*# of Test Files)
    if len(proj_kp_lines) != (NUM_POINTS + 1)*len(test_set):
        raise Exception('KP file length is not (NUM_POINTS + 1) x the number of loaded test files!')

    # For Each Image, Create Dictionary with The Image's:
    # Width, Height, True Pose [X,Y,Z,ZA,XA,YA], Calibration File Info [PD,XO,YO,PP], 
    # Model Key Points [NUM_POINTS by [X,Y,Z]], Projected Key Points [NUM_POINTS by [X,Y]]
    filtered_image_info = []
    print('Test set has', len(test_set),'images!')
    for image_indx in range(len(test_set)):
        # Image Width and Height
        img_w = int(info_lines[image_indx*13 + 9].split(':')[1].strip())
        img_h = int(info_lines[image_indx*13 + 10].split(':')[1].strip())
        # True Pose
        true_pose_str = info_lines[image_indx*13 + 11].split(':')[1].strip().split(',')
        true_pose = [float(i) for i in true_pose_str]
        # Get Image Home Directory
        img_home_dir = os.path.dirname(info_lines[image_indx*13].split('Path:')[1].strip())
        # Calibration Info
        calib_info_str = [line.strip() for line in open(img_home_dir + '/calibration.txt')][1:5]
        calib_info = [float(i) for i in calib_info_str]
        # Model Key Points (NUM_POINTS by (X,Y,Z))
        model_kp_whole_file = [line.rstrip('\n') for line in open(img_home_dir + '/'+ MODEL_TYPE +'.kp')]
        # Check there are 2 + NUM_POINTS LINES
        if len(model_kp_whole_file) != 2 + NUM_POINTS:
            raise Exception('Model key points file is not the right length!')
        model_kp_str = [line.split(':')[1].strip().split(',') for line in model_kp_whole_file[1:NUM_POINTS+1]]
        model_kp = [[float(j) for j in i] for i in model_kp_str]
        # Projected Key Points
        proj_kp_str = [line.strip().split(',') for line in proj_kp_lines[image_indx*(NUM_POINTS+1) + 1:\
                                                                          (image_indx*(NUM_POINTS+1) + NUM_POINTS + 1)]]
        proj_kp = [[float(j) for j in i] for i in proj_kp_str]
        # Store in Dictionary
        filtered_image_info.append({'image_width': img_w, 'image_height': img_h,'true_pose': true_pose,'calibration': calib_info,\
                                 'model_kp': model_kp, 'projected_kp': proj_kp})
    
    
    END READ IN IMAGE INFORMATION
    '''
    # Loop: predict, print and save
    model.eval()
    images_saved = 0
    total_test_loss = 0
    output_storage = np.empty([len(test_set), NUM_POINTS, 2])
    test_index_counter = 0
    for test_batch in test_generator:
        # Transfer to GPU
        test_batch, test_batch_labels = test_batch['image'].\
        to(main_device, dtype=torch.float, non_blocking=True),\
        test_batch['label'].to(main_device, dtype=torch.float, non_blocking=True)

        # Forward through the model
        test_output = model(test_batch)
        
        # Calculate loss and keep a running sum total of the loss over all the test data
        test_loss = loss_fn(test_output, test_batch_labels)
        total_test_loss += test_loss.item()*len(test_batch)
        
        # Print out image, predicted segmentation mask, and an overlay
        plot_predictions(test_batch,test_batch_labels, test_output,VALIDATION_BATCH_SIZE)

        # Store Outputs
        # Convert Heatmaps to Points By Taking The Maximum For Each Keypoint (Labels)
        heatmaps_output = test_output.cpu().detach()
        for tb_ind in range(len(test_batch)):
            KP_loc_guess = []
            for kp_ind in range(NUM_POINTS):
                m_ind = torch.argmax(heatmaps_output[tb_ind][kp_ind])
                x_ind = m_ind.item() % heatmaps_output[tb_ind][kp_ind].shape[1]
                y_ind = m_ind.item() // heatmaps_output[tb_ind][kp_ind].shape[1] # Indexed by top left corner (same as OpenCV)
                KP_loc_guess.append([x_ind/IMAGE_WIDTH, (IMAGE_HEIGHT - y_ind)/IMAGE_HEIGHT])
            output_storage[test_index_counter+tb_ind,:,:] = KP_loc_guess
         
        test_index_counter += len(test_batch)
    # Print out average test batch loss
    average_test_loss = total_test_loss/len(test_set)
    print('Average loss over test set:',average_test_loss)
   
    

## 2D Predicted Points Output
Writes the 2D predicted points for each test image to a single file.

In [ ]:
# Must Create New Output File
file = open(DATA_HOME_DIR + "/" + "predicted_points.txt", "w")
for data_idx, data_image in enumerate(output_storage):
    file.write("Image_" + str(data_idx) + "\n")
    for out_point in data_image:
        file.write(str(out_point[0]) + "," + str(out_point[1]) + "\n")
file.close()

## Kinematics Predictor
Predicts kinematics using PNP solvers on various subsets of the data predicted from the neural network. For each image in the test set, a pose is guessed and displayed. The actual true pose, the absolute error, and the running average of the absolute error are also displayed.

In [ ]:
# For Each Frame, Run Metrics Like Which Points Are The Most Accurate
kinematics_storage = np.empty([len(test_set), 6]) # [T_X, T_Y, T_Z, R_Z, R_X, R_Y]
point_guess_L2_Error_storage = [] # L2 Error stored for each of the NUM_Points
for num_points in range(NUM_POINTS):
    point_guess_L2_Error_storage.append([])
kinematic_L1_error_storage = [] # L1 Error stored for each of the kinematics poses
for pose_dof in range(6):
    kinematic_L1_error_storage.append([])
kp_used_storage = [] # Stores each time a key point is used in an ideal kinematic
for test_image_idx, test_image in enumerate(test_set):
    
    #Print out Test Image For Viewing
    #plt.figure(figsize=(16,(IMAGE_HEIGHT/IMAGE_WIDTH)*16))
    #I = test_image['image'].to("cpu").type(torch.float32)
    #img_PIL = torchvision.transforms.ToPILImage()(I.type(torch.uint8))
    #plt.imshow(img_PIL)
    #plt.show()
    
    # Convert Guesses for Each Point in Image to Tuples That Have been Unnormalized to
    #  ORIGINAL Image Dimensions (Not Necessarily Training Image Dimensions) Which Have Been Read In
    original_image_width = filtered_image_info[test_image_idx]['image_width']
    original_image_height = filtered_image_info[test_image_idx]['image_height']
    image_points_guesses = np.array([tuple([output_storage[test_image_idx,point,0] * original_image_width,\
                           output_storage[test_image_idx,point,1] * original_image_height]) for point in range(NUM_POINTS)])
    # Read in Calibration File for Image
    principal_distance = filtered_image_info[test_image_idx]['calibration'][0]
    principal_x = filtered_image_info[test_image_idx]['calibration'][1]
    principal_y = filtered_image_info[test_image_idx]['calibration'][2]
    pixel_pitch = filtered_image_info[test_image_idx]['calibration'][3]

    # Add L2 Point Errors
    for point_indx in range(NUM_POINTS):
        point_guess_L2_Error_storage[point_indx].append(math.sqrt(\
                  (pixel_pitch*(image_points_guesses[point_indx][0] -\
                                filtered_image_info[test_image_idx]['projected_kp'][point_indx][0]* original_image_width))**2 +\
                  (pixel_pitch*(image_points_guesses[point_indx][1] -\
                                filtered_image_info[test_image_idx]['projected_kp'][point_indx][1]* original_image_height))**2))

    # Read in Correct Location(s) of Points for Image
    model_points = np.array([tuple(mod_kp) for mod_kp in filtered_image_info[test_image_idx]['model_kp']], dtype="double")

    # Create Camera Matrix For Image
    camera_matrix = np.array([
                                [-1.0*principal_distance/pixel_pitch, 0, original_image_width/2.0 - principal_x/pixel_pitch],
                                [0, -1.0*principal_distance/pixel_pitch, original_image_height/2.0 - principal_y/pixel_pitch],
                                [0, 0, 1]
                            ], dtype = "double")

    # Solve PNP for All Combinations of Size NUM_GUESSES_FOR_PNP from the image_points_guesses
    # The one with the least reprojection error will be taken as the estimate pose
    NUM_GUESSES_FOR_PNP = NUM_POINTS   
    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
    min_reproj_err_val = math.inf
    arg_min_val_indices = []
    for comb in itertools.combinations(range(NUM_POINTS), NUM_GUESSES_FOR_PNP):
        # Solve PNP
        (success, rotation_vector, translation_vector, inliers) = cv2.solvePnPRansac(\
                                                                      model_points[list(comb)],\
                                                                      image_points_guesses[list(comb)],\
                                                                      camera_matrix, dist_coeffs,\
                                                                      reprojectionError=20.0)

        # Get Reprojection Error
        projected_PNP_guesses, _ = cv2.projectPoints(\
                                                     model_points[list(comb)],\
                                                     rotation_vector,\
                                                     translation_vector,\
                                                     camera_matrix, dist_coeffs)
        projected_PNP_guesses = np.squeeze(projected_PNP_guesses)
        reproj_error = ((projected_PNP_guesses - image_points_guesses[list(comb)])**2).mean()

        # Convert to RzRxRy Euler Angles
        R = cv2.Rodrigues(rotation_vector)[0]
        if (np.asscalar(translation_vector[2]) > 0): # Flip to correct side of camera by multiplying R and T by -1
            translation_vector = -1*translation_vector
            R[0:2,:] =-1*R[0:2,:]
        theta_x = math.asin(R[2,1])
        if (theta_x < math.pi/2):
            if (theta_x > -1*math.pi/2):
                theta_z = math.atan2(-1*R[0,1],R[1,1])
                theta_y = math.atan2(-1*R[2,0],R[2,2])
            else:
                # Not a unique solution
                theta_z = -1*math.atan2(R[0,2],R[0,0])
                theta_y = 0
        else:
            # Not a unique solution
            theta_z = math.atan2(R[0,2],R[0,0])
            theta_y = 0

        if (reproj_error < min_reproj_err_val):
            min_reproj_err_val = reproj_error
            rot_z = 360*theta_z/(2*math.pi)
            if rot_z < -180:
                rot_z += 360
            if rot_z > 180:
                rot_z -= 360
            rot_x = 360*theta_x/(2*math.pi)
            if rot_x < -180:
                rot_x += 360
            if rot_x > 180:
                rot_x -= 360
            rot_y = 360*theta_y/(2*math.pi)
            if rot_y < -180:
                rot_y += 360
            if rot_y > 180:
                rot_y -= 360
            kinematic_guess_for_image = [ np.asscalar(translation_vector[0]), np.asscalar(translation_vector[1]), np.asscalar(translation_vector[2]),
                                                rot_z,
                                                rot_x,
                                                rot_y]
            arg_min_val_indices = tuple(indices[0] for indices in inliers)
    print(kinematic_guess_for_image)
    print(filtered_image_info[test_image_idx]['true_pose'])
    for pose_dof in range(6):
        L1_abs_err = abs(filtered_image_info[test_image_idx]['true_pose'][pose_dof] \
                                                       - kinematic_guess_for_image[pose_dof])
        if pose_dof > 2: # IF ROTATION
            if L1_abs_err > 180:
                L1_abs_err = 360 - L1_abs_err
        kinematic_L1_error_storage[pose_dof].append(L1_abs_err)
    print(np.mean(kinematic_L1_error_storage, axis=1))
    kp_used_storage.append(arg_min_val_indices)
    print("--"*20)
    print("--"*20)

    # Store Kinematics
    kinematics_storage[test_image_idx,:] = kinematic_guess_for_image


# Histogram KP Error
Histograms of key point error in-plane for each of the NUM_POINTS key points.

In [ ]:
# Plot Histograms of Each Key Point Error
for data_idx, data in enumerate(point_guess_L2_Error_storage):
    # fixed bin size
    bins = np.arange(0, 100, 1) # fixed bin size
    plt.xlim([min(data)-5, max(data)+5])
    #plt.xlim([0, 20])
    plt.hist(data, bins=bins, alpha=0.5)
    plt.title('Histogram of L2 Error (mm) for Point %i' %(data_idx+1))
    plt.xlabel('L2 Error (bin size = 1)')
    plt.ylabel('Count')
    plt.show()

# Histogram DOF Error
Histograms that plot the absolute error in mm/deg for each of the degrees of freedom.

In [ ]:
# Plot Histograms of DOF Error
for data_idx, data in enumerate(kinematic_L1_error_storage):
    # fixed bin size
    bins = np.arange(0, 1000, 1) # fixed bin size
    plt.xlim([min(data)-5, max(data)+5])
    #plt.xlim([0, 20])
    plt.hist(data, bins=bins, alpha=0.5)
    plt.title('Histogram of Absolute Error Chart %i' %(data_idx+1))
    plt.xlabel('Absolute Error (bin size = 1)')
    plt.ylabel('Count')
    plt.show()

# Histogram of Number of Times a KP Used in Guess for Ideal Pose
Histogram of the number of times each key point (indexed from zero) was used in the guess for the true pose.

In [ ]:
data = list(itertools.chain(*kp_used_storage))
# fixed bin size
bins = np.arange(0, 1000, 1) # fixed bin size
plt.xlim([min(data), max(data) + 1])
plt.hist(data, bins=bins, alpha=0.5)
plt.title('Histogram of Number of Times a Key Point Was Used in an Ideal Kinematic')
plt.xlabel('Key Point Index')
plt.ylabel('Count')
plt.show()

# Output Kinematics File
Writes an estimated kinematics file (.jtak) for each study and places it in the correct directory.

In [ ]:
# Parse Info Lines To Get Image Paths
image_paths_from_info_file = info_lines[0::13]

# Sort Alphabetically and Get Indices
sorted_image_paths = np.sort(image_paths_from_info_file)
arg_sorted_image_paths = np.argsort(image_paths_from_info_file)

# Parse Sorted Image Paths to Get Study Directories
sorted_directories = [os.path.dirname(info[12:]) for info in sorted_image_paths]

# Create and Fill Kinematics Estimate Files
prev_dir = ""
SIG_FIGS = 4
for indx_sd, test_img in enumerate(sorted_directories):
    if  prev_dir != test_img:
        if prev_dir != "":
            file.close()
        prev_dir = test_img
        # Must Create New Kinematics File
        file = open(test_img + "/" + MODEL_TYPE + "_estimated_kin_" + MODEL_NAME + ".jtak", "w")
        file.write("JTA_EULER_KINEMATICS\nX_TRAN\t\tY_TRAN\t\tZ_TRAN\t\tZ_ROT\t\tX_ROT\t\tY_ROT\n")
    est_kin = kinematics_storage[arg_sorted_image_paths[indx_sd]]
    file.write(str(round(est_kin[0],4)) + ",\t\t" + str(round(est_kin[1],4)) + ",\t\t" + str(round(est_kin[2],4))\
               + ",\t\t" + str(round(est_kin[3],4)) + ",\t\t"  + str(round(est_kin[4],4)) + ",\t\t" + str(round(est_kin[5],4)) + ",\n")

## Delete Kinematics Files
Deletes estimated kinematics files from folders once finished (so as to not interfere with the Study to Grid writer).

In [ ]:
# Deletes Kinematics Estimate Files
prev_dir = ""
for indx_sd, test_img in enumerate(sorted_directories):
    if  prev_dir != test_img:
        if prev_dir != "":
            file.close()
        prev_dir = test_img
        # Must Create New Kinematics File
        os.remove(test_img + "/" + MODEL_TYPE + "_estimated_kin_" + MODEL_NAME + ".jtak")